In [3]:
import pandas as pd, re

xlsx_path = "Disciplinas_2s_2025V2.xlsx"           # caminho do arquivo

# ------------ utilitário: curso / semestre a partir do nome da aba -----------
def parse_course_sem(sheet_name):
    course, _, rest = sheet_name.rpartition("_")
    course = (course or sheet_name).replace("_", " ").strip()
    m = re.search(r"(\d+)", rest)
    semester = int(m.group(1)) if m else None
    return course, semester

# ------------ encontra a linha que contém “Segunda / Terça / …” --------------
weekdays = {"Segunda", "Terça", "Quarta", "Quinta", "Sexta", "Sábado"}
def find_header_row(df, max_search=10):
    """
    Procura (nas primeiras `max_search` linhas) por uma linha que tenha
    pelo menos 3 nomes de dias da semana.  Devolve o índice ou None.
    """
    for idx in range(min(max_search, len(df))):
        vals = {str(v).strip() for v in df.iloc[idx] if isinstance(v, str)}
        if len(vals & weekdays) >= 3:
            return idx
    return None

# ------------ extrai as aulas de UMA aba -------------------------------------
def extract_records(df):
    if df.empty:
        return []                         # aba sem dados

    header_row = find_header_row(df)
    if header_row is None:
        return []                         # estrutura não reconhecida

    # mapa dia  → 1ª coluna (início) do par
    day_cols = {
        day.strip(): col
        for col, day in df.iloc[header_row].items()
        if isinstance(day, str) and day.strip() in weekdays
    }

    # blocos sempre começam com a string “Faixa horária” na 1ª coluna
    starts = [
        idx for idx, val in enumerate(df[0])
        if isinstance(val, str) and val.strip().lower().startswith("faixa")
    ]

    recs = []
    for r in starts:
        time_row, code_row = df.iloc[r], df.iloc[r + 1]
        room_row, teacher_row = df.iloc[r + 2], df.iloc[r + 3]

        for dia, col in day_cols.items():
            codigo = code_row[col]
            if pd.isna(codigo) or str(codigo).strip() == "":
                continue

            # horário: col (início) e col+1 (fim) — verifique se col+1 existe
            fmt = lambda t: t.strftime("%H:%M") if hasattr(t, "strftime") else str(t)
            inicio = fmt(time_row[col])
            fim = fmt(time_row[col + 1]) if col + 1 in df.columns else None

            recs.append({
                "Disciplina": str(codigo).strip(),
                "Dia": dia,
                "Início": inicio,
                "Fim": fim,
                "Sala": None if pd.isna(room_row[col]) else str(room_row[col]).strip(),
                "Docente": None if pd.isna(teacher_row[col]) else str(teacher_row[col]).strip(),
            })
    return recs

# ------------ percorre TODAS as abas -----------------------------------------
excel = pd.ExcelFile(xlsx_path)
all_rows = []

for sheet in excel.sheet_names:
    raw = pd.read_excel(excel, sheet_name=sheet, header=None)
    curso, semestre = parse_course_sem(sheet)

    for rec in extract_records(raw):
        rec["Curso"] = curso
        rec["Semestre"] = semestre
        all_rows.append(rec)

grade_df = pd.DataFrame(all_rows).sort_values(
    ["Curso", "Semestre", "Dia", "Início"]
)

# Exibe ou salva:
grade_df.head()          # mostra primeiras linhas
grade_df.to_csv("grade_completa.csv",
                index=False,
                encoding="utf-8-sig")   # <--  BOM UTF-8: Excel reconhece

